<h1><center style="color:#2B3698">Merging the different methods for generating matching keywords</center></h1>

Importing the different libraries

## Merging the different methods

In [1]:
categories  = [
    'advice',
    'hygiene',
    'equipment',
    'activities',
    'technology',
    'info',
    'administrative',
    'job',
    'education',
    'home',
    'health',
    'food'
]

Hypothesis : our input will be supposed to be a dictionnary of the form {"name_category" : {"matching_keywords" : {"Model" : score}}} and the return would be {"name_category" : {"matching_keywords" : score}}

Question : Deal with stems or all words ?
Hypothesis : we will keep only stems in our inputs and then generate the lems afterward.

### TO DO : create a function generate_input(models, name_models, category) and return an input

##### TO DO : generate a function that takes as input a word and returns matching keywords and scores using wordnet

In [24]:
from nltk.corpus import wordnet as wn
from stemming.porter2 import stem
from nltk.corpus import wordnet_ic

In [28]:
def score_wordnet(word, matching_keyword) :
    word = wn.synsets(word)[0]
    matching_keyword = wn.synsets(matching_keyword)[0]
    score = 0
    try :
        score += word.path_similarity(matching_keyword)
        score += word.lch_similarity(matching_keyword)
        score += word.wup_similarity(matching_keyword)
    except :
        pass
    # TO DO add an other argument to the function (needs 3 arguments)
    # score += word.jcn_similarity(matching_keyword) 
    
    return score

In [29]:
def generate_words_wordnet(word, n = 3) :
    """
    Parameters
    ----------
    word: word to compute similarities to (can be a category word)
    n: number of similar words to get
    """
    
    name_model = "wordnet"
    result = {}
    
    for synset in wn.synsets(word):
        print(word)
        for lemma in synset.lemma_names():
            result[stem(lemma)] = {name_model : score_wordnet(word, lemma)}
    
    
    for i in range(n) :
        for origin in result.keys():
            print(origin)
            for synset in wn.synsets(origin):
                for lemma in synset.lemma_names():
                    result[stem(lemma)] = {name_model : score_wordnet(word, lemma)}
        
        
    return result

In [30]:
generate_words_wordnet("food")

food
food
food
food
nutrient
food_for_thought
solid_food
intellectual_nourish
nourish
nutrit
aliment
food
solid_food
alimentari
nutriti
intellectual_nourish
nutrient
food_for_thought
nourish
nutrit
aliment
food
susten
nutriment
solid_food
alimentari
nutriti
victual
sustain
nurtur
intellectual_nourish
nutrient
food_for_thought
nutrifi


{u'affirm': {'wordnet': 0},
 u'aliment': {'wordnet': 0},
 u'alimentari': {'wordnet': 0},
 u'comest': {'wordnet': 4.353529888257349},
 u'confirm': {'wordnet': 0},
 u'corrobor': {'wordnet': 0},
 u'eatabl': {'wordnet': 4.353529888257349},
 u'edibl': {'wordnet': 4.353529888257349},
 u'food': {'wordnet': 5.637586159726386},
 u'food_for_thought': {'wordnet': 1.6168192485505222},
 u'get': {'wordnet': 0.9743074561930456},
 u'have': {'wordnet': 2.2345331535282154},
 u'hold': {'wordnet': 1.2924170220450686},
 u'hold_up': {'wordnet': 0},
 u'intellectual_nourish': {'wordnet': 1.6168192485505222},
 u'keep': {'wordnet': 1.4972666445037728},
 u'keep_up': {'wordnet': 0},
 u'maintain': {'wordnet': 0},
 u'nourish': {'wordnet': 0},
 u'nurtur': {'wordnet': 1.6168192485505222},
 u'nutrient': {'wordnet': 5.637586159726386},
 u'nutrifi': {'wordnet': 0},
 u'nutriment': {'wordnet': 4.353529888257349},
 u'nutrit': {'wordnet': 0},
 u'nutriti': {'wordnet': 0},
 u'pabulum': {'wordnet': 4.353529888257349},
 u'prolo

##### TO DO : generate a function that takes as input a word and returns matching keywords and scores using word2vec

In [ ]:
def most_similar_word2vec(word, n, path, model_name = "text8-vector.bin"):
    """ Most similar words to word and their scores.
    Parameters
    ----------
    word: word to compute similarities to (can be a category word)
    n: number of similar words to get
    path: path of the pretrained word2vec model
    model_name: name of the pretrained word2vec model

    Returns
    -------
    d: {word: {similar_word: {word2vec: word2vec_score}}

    barplot: single bar plot
    """
    d = {}
    # Load Google's pre-trained Word2Vec model
    model = gensim.models.KeyedVectors.load_word2vec_format(os.path.join(path, "text8-vector.bin"), binary=True) 
    d[word] = {key: {"word2vec": value} for (key, value) in model.most_similar("food", topn = n)}
    return d

##### TO DO : generate a function that takes as input a word and returns matching keywords and scores using spacy

# Sources

http://ftp.cs.toronto.edu/pub/gh/Budanitsky+Hirst-2001.pdf

# Documentation (error)

import error : https://stackoverflow.com/questions/15526996/ipython-notebook-locale-error

In [ ]:
from nltk.corpus import wordnet as wn

In [ ]:
!pip install stemming

In [ ]:
from stemming.porter2 import stem

In [ ]:
origin = "food"

In [ ]:
wn.synsets(origin)

In [ ]:
l = []
for synset in wn.synsets(origin):
    for lemma in synset.lemma_names():
        l.append(stem(lemma))

In [ ]:
for origin in l :
    for synset in wn.synsets(origin):
        for lemma in synset.lemma_names():
            l.append(stem(lemma))
    

In [ ]:
l